In [ ]:
%matplotlib inline

import numpy as np
from argparse import Namespace
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import phate

# Prepared embedded data

In [ ]:
"""load saved trajectories data for npz file
"""
SEQ = "PT4"
# SEQ = "PT4_hairpin"

# laod pre-training data
fnpz_data = "./data/pretraining/pretraining_{}.npz".format(SEQ)
data_npz = np.load(fnpz_data)

# asssign data to variables
for var in data_npz.files:
     locals()[var] = data_npz[var]
     print(var, locals()[var].shape)

# # recover full data based on coord_id, indices, and unique data
# SIMS_adj = SIMS_adj_uniq[coord_id_S]
# SIMS_scar = SIMS_scar_uniq[coord_id_S]
# SIMS_G = SIMS_G_uniq[coord_id_S]
# SIMS_pair = SIMS_pair_uniq[coord_id_S]

# print(SIMS_T.shape,SIMS_HT.shape,SIMS_HT_uniq.shape)
# print(SIMS_adj.shape,SIMS_scar.shape,SIMS_G.shape,SIMS_HT.shape,SIMS_pair.shape)
# print(SIMS_adj_uniq.shape,SIMS_scar_uniq.shape,SIMS_G_uniq.shape,SIMS_pair_uniq.shape) 
# print(SIMS_dict.shape,SIMS_dict_uniq.shape)
# print(coord_id_S.shape,indices_S.shape,trj_id.shape,occ_density_S.shape)

In [ ]:
X_j = np.array(np.load(f'./data/graph/{SEQ}/shortestpath_knn=100.npz',allow_pickle=True)["X_j"], dtype=int)
D_ij = np.array(np.load(f'./data/graph/{SEQ}/shortestpath_knn=100.npz',allow_pickle=True)["D_ij"], dtype=float)
print("X_j", X_j.shape)
print("D_ij", D_ij.shape)

In [ ]:
# load embedding plot data

fnpz_data_embed = f"./data/vida_data/{SEQ}_0304-0216.npz"
# fnpz_data_embed = f"./data/vida_data/{SEQ}_usePT4_03040216.npz"
data_npz_embed = np.load(fnpz_data_embed,allow_pickle=True)
# asssign data to variables
for var in data_npz_embed.files:
    globals()[var] = data_npz_embed[var]
    print(var, globals()[var].shape)
print("data_embed", data_embed.max(), data_embed.min(), data_embed.mean(), data_embed.std())

In [ ]:
# calculate the probability of being visited during a simulated trajectory 
# from the initial state
split_id = trj_id + 1 # index for split to each trajectory
P_tot = np.zeros(len(SIMS_dict_uniq))

for i in range(len(split_id)):
    if i == 0:
        trj = set(SIMS_dict[0:split_id[i],4].astype(int))
    else:
        trj = set(SIMS_dict[split_id[i-1]:split_id[i],4].astype(int))

    P_tot[list(trj)] += 1

P_tot = P_tot / 100

print("P_tot", P_tot.shape, P_tot.max(), P_tot.min())

# Evaluate embedding

### Metric distance

In [ ]:
def metric_dist(X_j, D_ij, P_tot, z):
    """
    Metric to calculate the distance 
    """
    z_re = z.reshape(-1,1,z.shape[-1])
    zj = z[X_j]
    l2_zizj = np.sqrt(np.sum((z_re-zj)**2, axis=-1))
    
    # normalize the distance
    scaler = MinMaxScaler(feature_range=(1,3)) 
    l2_zizj = scaler.fit_transform(l2_zizj)
    D_ij = scaler.fit_transform(D_ij)
    
    dist_diff = (l2_zizj - D_ij)**2
    wij = (P_tot.reshape(-1,1) * P_tot[X_j])
    dist_loss = np.sum(wij * dist_diff)/len(dist_diff)
    return dist_loss

In [ ]:
pca_dist = metric_dist(X_j, D_ij, P_tot, pca_coords[:,:2])
phate_dist = metric_dist(X_j, D_ij, P_tot, phate_coords)
print (f'PCA distance loss: {pca_dist:.4f}')
print (f'PHATE distance loss: {phate_dist:.4f}')

### Neighboring preservation rate

In [ ]:
from sklearn.neighbors import NearestNeighbors

def neighboring_preservation_rate(X, X_j, k):
    """
    Metric to calculate the neighboring preservation rate 
    """
    # Compute the k-nearest neighbors for both X and Y.
    nn_X = NearestNeighbors(n_neighbors=k+1).fit(X) # k+1 because we don't want to include the point itself
    indices_X = nn_X.kneighbors(X,return_distance=False)[:,1:] # exclude the point itself
    
    # compute the rate of each point
    rate_list = []
    for i in range(len(indices_X)):
        count = len(np.intersect1d(indices_X[i], X_j[i,:k]))
        rate_i = count/k
        rate_list.append(rate_i)
        
    # Compute the overall neighboring preservation rate
    return np.mean(rate_list)

In [ ]:
knnn = 100
print("PCA rate: ", neighboring_preservation_rate(pca_coords[:,:2], X_j, k=knnn))
print("PHATE rate: ", neighboring_preservation_rate(phate_coords, X_j, k=knnn))

In [ ]:
print("Direct PCA rate: ", neighboring_preservation_rate(pca_coords_direct[:,:2], X_j, k=knnn))
print("Direct PHATE rate: ", neighboring_preservation_rate(phate_coords_direct, X_j, k=knnn))

### PCA explained variance

In [ ]:
cm = PCA(n_components=25)
cm.fit(data_embed)

PC_values = np.arange(cm.n_components_) + 1
plt.plot(PC_values, np.cumsum(cm.explained_variance_ratio_), 'ro-', linewidth=2)
plt.title('Scree Plot: PCA')
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance');
# plt.xticks(np.arange(0, data_embed.shape[-1]+1, 1))

plt.show()

In [ ]:
np.cumsum(cm.explained_variance_ratio_)

# Visualize

In [ ]:
SEQ

### PCA Vis

In [ ]:
%matplotlib inline
X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

# annotations=["I","F"]
# x = [X[0],X[all_nodes[-1]]]
# y = [Y[0],Y[all_nodes[-1]]]
# plt.scatter(x,y,s=150, c="green", alpha=1)
# for i, label in enumerate(annotations):
#     plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="yellow")

#### Try use PCA directly without AE

In [ ]:
# %%script false --no-raise-error

pca_coords_direct = PCA(n_components=3).fit_transform(SIMS_scar_uniq)   # multiple trj

X = pca_coords_direct[:,0]
Y = pca_coords_direct[:,1]
Z = pca_coords_direct[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="black")

In [ ]:
cm = PCA(n_components=25)
cm.fit(SIMS_scar_uniq)

PC_values = np.arange(cm.n_components_) + 1
plt.plot(PC_values, np.cumsum(cm.explained_variance_ratio_), 'ro-', linewidth=2)
plt.title('Scree Plot: PCA')
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance');
# plt.xticks(np.arange(0, data_embed.shape[-1]+1, 1))

plt.show()

In [ ]:
np.cumsum(cm.explained_variance_ratio_)

### PHATE Vis

In [ ]:
X_phate = phate_coords[:,0]
Y_phate = phate_coords[:,1]

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X_phate,Y_phate,
                c=SIMS_G_uniq,            
                cmap='plasma',
               )

plt.colorbar(im)

# annotations=["I","F"]
# x = [X_phate[0],X_phate[all_nodes[-1]]]
# y = [Y_phate[0],Y_phate[all_nodes[-1]]]
# plt.scatter(x,y,s=50, c="green", alpha=1)
# for i, label in enumerate(annotations):
#     plt.annotate(label, (x[i],y[i]),fontsize=30,c="black")

#### PHATE without AE

In [ ]:
# %%script false --no-raise-error

phate_operator = phate.PHATE(n_jobs=-2)
phate_coords_direct = phate_operator.fit_transform(SIMS_scar_uniq)

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(phate_coords_direct[:,0],
          phate_coords_direct[:,1],
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [phate_coords_direct[:,0][0],phate_coords_direct[:,0][-1]]
y = [phate_coords_direct[:,1][0],phate_coords_direct[:,1][-1]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=20,c="black")